<div style="text-align:center">
    <h1>Advanced Retrievers in LlamaIndex</h1>
</div>

# Setup

In [1]:
import os
import json
from typing import List, Optional
import asyncio
import warnings
import numpy as np
warnings.filterwarnings('ignore')

# Core LlamaIndex imports
from llama_index.core import (
    VectorStoreIndex, 
    SimpleDirectoryReader, 
    Document,
    Settings,
    DocumentSummaryIndex,
    KeywordTableIndex
)
from llama_index.core.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
    AutoMergingRetriever,
    RecursiveRetriever,
    QueryFusionRetriever
)
from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexLLMRetriever,
    DocumentSummaryIndexEmbeddingRetriever,
)
from llama_index.core.node_parser import SentenceSplitter, HierarchicalNodeParser
from llama_index.core.schema import NodeWithScore, QueryBundle
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.embeddings import BaseEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Advanced retriever imports
from llama_index.retrievers.bm25 import BM25Retriever

from llama_index.llms.ollama import Ollama

# Sentence transformers
from sentence_transformers import SentenceTransformer

# Statistical libraries for fusion techniques
try:
    from scipy import stats
    SCIPY_AVAILABLE = True
except ImportError:
    SCIPY_AVAILABLE = False
    print("⚠️ scipy not available - some advanced fusion features will be limited")

print("✅ All imports successful!")

2026-01-10 11:45:13.969237: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


✅ All imports successful!


In [2]:
def ollama_llm(max_tokens=500):
    ollama_llm = Ollama(
        model='llama3.2:latest',
        request_timeout=120.0,
        temperature=0.0,
        max_output_tokens=max_tokens,
        context_window=2000,           # newer llama3.2 supports more
        format="json",               # ← try this if your ollama version supports it
    )
    return ollama_llm

In [3]:
# Initialize embedding model first
print("🔧 Initializing HuggingFace embeddings...")
embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5", device = 'cpu'
)
print("✅ HuggingFace embeddings initialized!")

# Setup with watsonx.ai
print("🔧 Initializing watsonx.ai LLM...")
llm = ollama_llm()

# Configure global settings
Settings.llm = llm
Settings.embed_model = embed_model
print("✅ watsonx.ai LLM and embeddings configured!")

🔧 Initializing HuggingFace embeddings...
✅ HuggingFace embeddings initialized!
🔧 Initializing watsonx.ai LLM...
✅ watsonx.ai LLM and embeddings configured!


In [4]:
# Sample data for the lab - AI/ML focused documents
SAMPLE_DOCUMENTS = [
    "Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn from data.",
    "Deep learning uses neural networks with multiple layers to model and understand complex patterns in data.",
    "Natural language processing enables computers to understand, interpret, and generate human language.",
    "Computer vision allows machines to interpret and understand visual information from the world.",
    "Reinforcement learning is a type of machine learning where agents learn to make decisions through rewards and penalties.",
    "Supervised learning uses labeled training data to learn a mapping from inputs to outputs.",
    "Unsupervised learning finds hidden patterns in data without labeled examples.",
    "Transfer learning leverages knowledge from pre-trained models to improve performance on new tasks.",
    "Generative AI can create new content including text, images, code, and more.",
    "Large language models are trained on vast amounts of text data to understand and generate human-like text."
]

# Consistent query examples used throughout the lab
DEMO_QUERIES = {
    "basic": "What is machine learning?",
    "technical": "neural networks deep learning", 
    "learning_types": "different types of learning",
    "advanced": "How do neural networks work in deep learning?",
    "applications": "What are the applications of AI?",
    "comprehensive": "What are the main approaches to machine learning?",
    "specific": "supervised learning techniques"
}

print(f"📄 Loaded {len(SAMPLE_DOCUMENTS)} sample documents")
print(f"🔍 Prepared {len(DEMO_QUERIES)} consistent demo queries")
for i, doc in enumerate(SAMPLE_DOCUMENTS[:3], 1):
    print(f"{i}. {doc}")
print("...")

📄 Loaded 10 sample documents
🔍 Prepared 7 consistent demo queries
1. Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn from data.
2. Deep learning uses neural networks with multiple layers to model and understand complex patterns in data.
3. Natural language processing enables computers to understand, interpret, and generate human language.
...


In [5]:
class AdvancedRetrieversLab:
    def __init__(self):
        print("🚀 Initializing Advanced Retrievers Lab...")
        self.documents = [Document(text=text) for text in SAMPLE_DOCUMENTS]
        self.nodes = SentenceSplitter().get_nodes_from_documents(self.documents)
        
        print("📊 Creating indexes...")
        # Create various indexes
        self.vector_index = VectorStoreIndex.from_documents(self.documents)
        self.document_summary_index = DocumentSummaryIndex.from_documents(self.documents)
        self.keyword_index = KeywordTableIndex.from_documents(self.documents)
        
        print("✅ Advanced Retrievers Lab Initialized!")
        print(f"📄 Loaded {len(self.documents)} documents")
        print(f"🔢 Created {len(self.nodes)} nodes")

# Initialize the lab
lab = AdvancedRetrieversLab()

🚀 Initializing Advanced Retrievers Lab...
📊 Creating indexes...
current doc id: b34ab465-4a91-40fd-a8e7-a40f59cfee94
current doc id: 364b2795-3ac1-4abb-a8be-c46310debed5
current doc id: 7d2ed37b-1f5b-4134-9cbf-af0f7f245dae
current doc id: 4542484b-7b5c-4e2b-9bc0-c0d22f7964f5
current doc id: 7df0a828-b3f7-4475-be57-87fe132cff3e
current doc id: cc44e17a-e4f0-48e0-be93-2ca554df19e9
current doc id: 750f7098-9eee-4f31-a9f9-aeffb2a837b0
current doc id: 9f32891d-6cd6-4732-b9de-49140148af5d
current doc id: 5e41b1ed-7746-4485-99c7-2fbc5c806413
current doc id: 049e276e-9b9f-47c1-be9a-d57df1951bcb
✅ Advanced Retrievers Lab Initialized!
📄 Loaded 10 documents
🔢 Created 10 nodes


# Core Retriever Demonstrations

## 1. Vector Index Retriever

The Vector-Based Retriever relies on vector embeddings to identify content that is semantically similar, making it the most widely used approach for general semantic search and the foundation of most modern retrieval-augmented generation (RAG) systems.

**How it operates:**
- Source documents are divided into smaller chunks (nodes)
- Each chunk is transformed into a dense vector using a selected embedding model
- The user's query is also converted into a vector
- The system returns the most relevant chunks, ranked according to cosine similarity between query and document vectors
- Embeddings are typically generated in reasonably large batches (often around 2000 items)

**Best suited for:**
- General semantic search tasks (the default choice in most applications)
- Discovering content that shares similar meaning or ideas
- RAG systems that depend on contextual understanding
- Situations where natural language understanding matters more than precise keyword matching

**Main characteristics:**
- Maintains vector representations for every document segment
- Excels at retrieving information based on conceptual similarity rather than literal text matches
- Serves as the standard retrieval mechanism in many LLM-powered applications

**Advantages:**
- Strong ability to capture meaning, nuance, and context
- Effectively handles synonyms, paraphrases, and conceptually related content
- Performs well with conversational or free-form natural language questions

**Limitations:**
- May overlook results that contain very specific keywords when exact phrasing is important
- Performance heavily depends on the quality of the embedding model being used
- Can become resource-intensive (memory and compute) when dealing with very large document sets

In [6]:
print("=" * 60)
print("1. VECTOR INDEX RETRIEVER")
print("=" * 60)

# Basic vector retriever
vector_retriever = VectorIndexRetriever(
    index=lab.vector_index,
    similarity_top_k=3
)

# Alternative creation method
alt_retriever = lab.vector_index.as_retriever(similarity_top_k=3)

query = DEMO_QUERIES["basic"]  # "What is machine learning?"
nodes = vector_retriever.retrieve(query)

print(f"Query: {query}")
print(f"Retrieved {len(nodes)} nodes:")
for i, node in enumerate(nodes, 1):
    print(f"{i}. Score: {node.score:.4f}")
    print(f"   Text: {node.text[:100]}...")
    print()

1. VECTOR INDEX RETRIEVER
Query: What is machine learning?
Retrieved 3 nodes:
1. Score: 0.8700
   Text: Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn fr...

2. Score: 0.7644
   Text: Reinforcement learning is a type of machine learning where agents learn to make decisions through re...

3. Score: 0.6979
   Text: Supervised learning uses labeled training data to learn a mapping from inputs to outputs....



## 2. BM25 Retriever – Precise Keyword Matching


BM25 is a powerful, statistically grounded keyword retrieval algorithm that significantly improves upon classic TF-IDF. It remains one of the most widely deployed ranking methods in modern search engines, including systems like Elasticsearch, Solr, and many enterprise search platforms.

### Quick Review: What TF-IDF Does (and Where It Falls Short)

TF-IDF (Term Frequency × Inverse Document Frequency) was a major advancement in information retrieval:

- **Term Frequency (TF)** — counts how frequently a term appears in a single document (usually normalized by document length)  
  Example: "algorithm" appearing 5 times in a 200-word document → TF = 5/200 = 0.025

- **Inverse Document Frequency (IDF)** — measures how rare or distinctive a term is across the entire collection  
  Example: If "algorithm" appears in 5 documents out of 10,000 → IDF = log(10,000/5) ≈ 7.6  
  Very common words (like "and", "is") receive very low IDF values

- **TF-IDF score** = TF × IDF  
  → Gives high importance to terms that are frequent in a document but rare overall

While effective, TF-IDF has two major practical weaknesses:
- Repeated occurrences of a term keep increasing the score without limit (no saturation)
- Longer documents tend to receive unfairly high scores simply because they contain more words

### How BM25 Solves These Problems

BM25 (Best Matching 25) introduces smart refinements that make it much more effective in real-world search:

1. **Term Frequency Saturation**  
   - Instead of letting term frequency grow linearly forever, BM25 applies a non-linear saturation curve  
   - Going from 1 to 10 occurrences gives a big score boost  
   - Going from 10 to 100 gives much less additional benefit  
   → Prevents very long, repetitive documents from dominating results

2. **Document Length Normalization**  
   - Adjusts scores based on how much longer or shorter a document is compared to the collection average  
   - Short documents aren’t unfairly penalized  
   - Very long documents don’t get an automatic advantage  
   - Controlled by parameter **b** (typically ~0.75)

3. **Tunable Hyperparameters**  
   - **k1** (~1.2–2.0) — controls how quickly term frequency saturation kicks in  
   - **b** (~0.75) — controls strength of length normalization (0 = none, 1 = full normalization)

### Best Use Cases for BM25

Choose BM25 when you need:

- High precision for exact keyword and phrase matching
- Searching technical manuals, API documentation, or code repositories
- Legal, medical, or patent documents where specific terminology is critical
- E-commerce product search with model numbers, SKUs, or precise attributes
- Any domain where users expect literal term matching over conceptual similarity

### Key Strengths

- Extremely fast ranking (no neural networks or heavy computation)
- Outstanding performance on exact-match and keyword-heavy queries
- No training data or model fine-tuning required
- Very interpretable — you can explain why a document ranked highly
- Battle-tested in production for decades

### Main Limitations

- Completely lacks semantic understanding (cannot match "car" ↔ "automobile" or "run" ↔ "jogging")
- Sensitive to spelling mistakes, word variations, and morphological differences
- No awareness of sentence structure, word order, or broader context
- Requires thoughtful tuning of k1 and b parameters for best results in each domain

In summary, BM25 remains the go-to choice whenever precise, fast, and reliable keyword-based retrieval is more important than semantic or conceptual matching.

In [7]:
print("=" * 60)
print("2. BM25 RETRIEVER")
print("=" * 60)

try:
    import Stemmer
    
    # Create BM25 retriever with default parameters
    bm25_retriever = BM25Retriever.from_defaults(
        nodes=lab.nodes,
        similarity_top_k=3,
        stemmer=Stemmer.Stemmer("english"),
        language="english"
    )
    
    query = DEMO_QUERIES["technical"]  # "neural networks deep learning"
    nodes = bm25_retriever.retrieve(query)
    
    print(f"Query: {query}")
    print("BM25 analyzes exact keyword matches with sophisticated scoring")
    print(f"Retrieved {len(nodes)} nodes:")
    
    for i, node in enumerate(nodes, 1):
        score = node.score if hasattr(node, 'score') and node.score else 0
        print(f"{i}. BM25 Score: {score:.4f}")
        print(f"   Text: {node.text[:100]}...")
        
        # Highlight which query terms appear in the text
        text_lower = node.text.lower()
        query_terms = query.lower().split()
        found_terms = [term for term in query_terms if term in text_lower]
        if found_terms:
            print(f"   → Found terms: {found_terms}")
        print()
    
    print("BM25 vs TF-IDF Comparison:")
    print("TF-IDF Problem: Linear term frequency scaling")
    print("  Example: 10 occurrences → score of 10, 100 occurrences → score of 100")
    print("BM25 Solution: Saturation function")
    print("  Example: 10 occurrences → high score, 100 occurrences → slightly higher score")
    print()
    print("TF-IDF Problem: No document length consideration")
    print("  Example: Long documents dominate results")
    print("BM25 Solution: Length normalization (b parameter)")
    print("  Example: Scores adjusted based on document length vs. average")
    print()
    print("Key BM25 Parameters:")
    print("- k1 ≈ 1.2: Term frequency saturation (how quickly scores plateau)")
    print("- b ≈ 0.75: Document length normalization (0=none, 1=full)")
    print("- IDF weighting: Rare terms get higher scores")
        
except ImportError:
    print("⚠️ BM25Retriever requires 'pip install PyStemmer'")
    print("Demonstrating BM25 concepts with fallback vector search...")
    
    fallback_retriever = lab.vector_index.as_retriever(similarity_top_k=3)
    query = DEMO_QUERIES["technical"]
    nodes = fallback_retriever.retrieve(query)
    
    print(f"Query: {query}")
    print("(Using vector fallback to demonstrate BM25 concepts)")
    
    for i, node in enumerate(nodes, 1):
        print(f"{i}. Vector Score: {node.score:.4f}")
        print(f"   Text: {node.text[:100]}...")
        
        # Demonstrate TF-IDF concept manually
        text_lower = node.text.lower()
        query_terms = query.lower().split()
        found_terms = [term for term in query_terms if term in text_lower]
        
        if found_terms:
            print(f"   → BM25 would boost this result for terms: {found_terms}")
        print()
    
    print("BM25 Concept Demonstration:")
    print("1. TF-IDF Foundation:")
    print("   - Term Frequency: How often words appear in document")
    print("   - Inverse Document Frequency: How rare words are across collection")
    print("   - TF-IDF = TF × IDF (balances frequency vs rarity)")
    print()
    print("2. BM25 Improvements:")
    print("   - Saturation: Prevents over-scoring repeated terms")
    print("   - Length normalization: Prevents long document bias")
    print("   - Tunable parameters: k1 (saturation) and b (length adjustment)")
    print()
    print("3. Real-world Usage:")
    print("   - Elasticsearch default scoring function")
    print("   - Apache Lucene/Solr standard")
    print("   - Used in 83% of text-based recommender systems")
    print("   - Developed by Robertson & Spärck Jones at City University London")

2. BM25 RETRIEVER
Query: neural networks deep learning
BM25 analyzes exact keyword matches with sophisticated scoring
Retrieved 3 nodes:
1. BM25 Score: 2.5203
   Text: Deep learning uses neural networks with multiple layers to model and understand complex patterns in ...
   → Found terms: ['neural', 'networks', 'deep', 'learning']

2. BM25 Score: 0.3372
   Text: Reinforcement learning is a type of machine learning where agents learn to make decisions through re...
   → Found terms: ['learning']

3. BM25 Score: 0.3024
   Text: Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn fr...
   → Found terms: ['learning']

BM25 vs TF-IDF Comparison:
TF-IDF Problem: Linear term frequency scaling
  Example: 10 occurrences → score of 10, 100 occurrences → score of 100
BM25 Solution: Saturation function
  Example: 10 occurrences → high score, 100 occurrences → slightly higher score

TF-IDF Problem: No document length consideration
  Example: Long docume

## 3. Document Summary–Based Retriever – Smart Document-Level Filtering


The Document Summary–Based Retriever is a clever two-stage approach designed for working efficiently with very large document collections.  
Instead of comparing the query directly against every full document, it first uses concise summaries to quickly identify the most promising documents — then returns the **complete original documents** (not the summaries).

### How It Works

1. **During indexing**:
   - Each full document is processed to create a short, meaningful summary (usually via an LLM)
   - Both the summary and the original document are stored
   - The summary is also converted into a vector embedding (for embedding-based retrieval)

2. **During retrieval**:
   - The system compares the user query against the **summaries only** (much faster than scanning full documents)
   - It selects the most relevant documents based on the summaries
   - Finally, it returns the **full, original versions** of those selected documents

This two-step filtering dramatically reduces the number of documents that need detailed processing, making it ideal for massive or diverse collections.

### Two Main Retrieval Modes

1. **LLM-Powered Summary Retriever** (`DocumentSummaryIndexLLMRetriever`)
   - An LLM evaluates the query against each document summary
   - Makes intelligent, context-aware decisions about relevance
   - Better suited for complex, nuanced, or multi-faceted questions
   - More accurate but slower and more expensive

2. **Embedding-Based Summary Retriever** (`DocumentSummaryIndexEmbeddingRetriever`)
   - Uses vector similarity between the query embedding and summary embeddings
   - Much faster and more cost-efficient
   - Excellent when semantic similarity between query and high-level document topic is sufficient

### Best Use Cases

This retriever shines when you are dealing with:

- Very large document sets (thousands to millions of documents)
- Collections containing many distinct topics or subject areas
- Multi-document question answering over heterogeneous sources
- Situations where loading every document into the LLM context window is impossible
- Enterprise knowledge bases, research paper archives, legal document repositories, technical manuals across many products, etc.

### Important Configuration Settings

- `choice_top_k` (for LLM mode) — how many documents the LLM should select
- `similarity_top_k` (for embedding mode) — how many most similar summaries to keep
- Default value is usually 1 → increase to 3–10 when you want multiple candidate documents

**Critical reminder**: The system **always returns the full original documents** — summaries are used only for fast filtering/selection.

### Key Advantages

- Greatly reduces the number of documents processed in detail
- Excellent at handling diverse, topic-scattered collections
- Preserves complete context and all original details in final results
- Very effective for scaling retrieval to huge corpora

### Main Limitations

- Requires generating summaries during indexing (usually needs LLM → adds time and cost at setup)
- Quality of retrieval depends heavily on how good the summaries are
- Some fine details or specific passages may be overlooked if not captured well in the summary
- LLM-based variant is slower and more expensive than pure vector or keyword methods

In short: use Document Summary–Based retrieval when you need to intelligently narrow down a huge, varied document collection before doing detailed matching — without ever losing access to the complete original content.

In [8]:
print("=" * 60)
print("3. DOCUMENT SUMMARY INDEX RETRIEVERS")
print("=" * 60)

# LLM-based document summary retriever
doc_summary_retriever_llm = DocumentSummaryIndexLLMRetriever(
    lab.document_summary_index,
    choice_top_k=3  # Number of documents to select
)

# Embedding-based document summary retriever  
doc_summary_retriever_embedding = DocumentSummaryIndexEmbeddingRetriever(
    lab.document_summary_index,
    similarity_top_k=3  # Number of documents to select
)

query = DEMO_QUERIES["learning_types"]  # "different types of learning"

print(f"Query: {query}")

print("\nA) LLM-based Document Summary Retriever:")
print("Uses LLM to select relevant documents based on summaries")
try:
    nodes_llm = doc_summary_retriever_llm.retrieve(query)
    print(f"Retrieved {len(nodes_llm)} nodes")
    for i, node in enumerate(nodes_llm[:2], 1):
        print(f"{i}. Score: {node.score:.4f}" if hasattr(node, 'score') and node.score else f"{i}. (Document summary)")
        print(f"   Text: {node.text[:80]}...")
        print()
except Exception as e:
    print(f"LLM-based retrieval demo: {str(e)[:100]}...")

print("B) Embedding-based Document Summary Retriever:")
print("Uses vector similarity between query and document summaries")
try:
    nodes_emb = doc_summary_retriever_embedding.retrieve(query)
    print(f"Retrieved {len(nodes_emb)} nodes")
    for i, node in enumerate(nodes_emb[:2], 1):
        print(f"{i}. Score: {node.score:.4f}" if hasattr(node, 'score') and node.score else f"{i}. (Document summary)")
        print(f"   Text: {node.text[:80]}...")
        print()
except Exception as e:
    print(f"Embedding-based retrieval demo: {str(e)[:100]}...")

print("Document Summary Index workflow:")
print("1. Generates summaries for each document using LLM")
print("2. Uses summaries to select relevant documents")
print("3. Returns full content from selected documents")

3. DOCUMENT SUMMARY INDEX RETRIEVERS
Query: different types of learning

A) LLM-based Document Summary Retriever:
Uses LLM to select relevant documents based on summaries
Retrieved 0 nodes
B) Embedding-based Document Summary Retriever:
Uses vector similarity between query and document summaries
Retrieved 3 nodes
1. (Document summary)
   Text: Reinforcement learning is a type of machine learning where agents learn to make ...

2. (Document summary)
   Text: Unsupervised learning finds hidden patterns in data without labeled examples....

Document Summary Index workflow:
1. Generates summaries for each document using LLM
2. Uses summaries to select relevant documents
3. Returns full content from selected documents


## 4. Auto-Merging Retriever – Smart Context Recovery for Long Documents


The Auto-Merging Retriever is a sophisticated retrieval strategy that solves one of the biggest problems in vector search: **loss of surrounding context** when documents are split into small chunks.

It automatically decides whether to return small, precise pieces or larger, more contextual sections — depending on how many related small pieces were found.

### Core Mechanism

1. **Hierarchical Chunking**  
   Documents are split into multiple levels of granularity:  
   - Large parent chunks (e.g., whole sections or pages)  
   - Smaller child chunks (e.g., paragraphs or sentences)

2. **Dual Storage Strategy**  
   - Only the **small child chunks** are embedded and indexed in the vector store → enables very precise semantic matching  
   - The **larger parent chunks** are kept in a separate document store (not embedded)

3. **Intelligent Merging Logic**  
   - When a query retrieves several child chunks  
   - If enough of those children belong to the same parent (threshold is configurable)  
   - → The system discards the individual children and returns the **complete parent chunk** instead

This behavior automatically gives you:  
→ Precise matching when only one small piece is relevant  
→ Richer context when multiple related pieces are triggered

### Best Use Cases

Choose the Auto-Merging Retriever when working with:

- Long-form content: research papers, technical manuals, books, annual reports  
- Legal contracts, patents, regulations — where context is critical  
- Documents that naturally have structure (chapters → sections → paragraphs)  
- Situations where small fixed-size chunks frequently cut important ideas in half  
- Applications that need both pinpoint accuracy and coherent, readable context

### Typical Configuration Options

- `chunk_sizes` — list of sizes from largest to smallest (example: `[1024, 512, 256]`)  
- `chunk_overlap` — number of tokens to overlap between chunks (helps continuity)  
- Merge threshold — minimum number of child chunks needed from the same parent to trigger merge (default often 2–3)  
- Vector store holds child nodes, document store holds parents

### Major Advantages

- Automatically recovers lost context without manual post-processing  
- Greatly reduces "fragmented answer" problem common in basic chunking  
- Keeps very fine-grained search precision when needed  
- Adapts dynamically to query complexity and match distribution  
- Works especially well on documents with logical hierarchy

### Main Drawbacks

- More complex implementation than simple vector or keyword retrievers  
- Requires thoughtful choice of multiple chunk sizes  
- Increased storage needs (multiple representations of the same content)  
- Less benefit on very short or completely flat documents  
- Initial indexing takes longer due to multiple chunking passes

**In essence**:  
The Auto-Merging Retriever gives you the best of both worlds — razor-sharp relevance from small chunks combined with the full surrounding context from larger sections — automatically deciding which level is most appropriate for each query.

https://docs.llamaindex.ai/en/stable/examples/retrievers/auto_merging_retriever/*

In [9]:
print("=" * 60)
print("4. AUTO MERGING RETRIEVER")
print("=" * 60)

# Create hierarchical nodes
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[512, 256, 128]
)

hier_nodes = node_parser.get_nodes_from_documents(lab.documents)

# Create storage context with all nodes
from llama_index.core import StorageContext
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.vector_stores import SimpleVectorStore

docstore = SimpleDocumentStore()
docstore.add_documents(hier_nodes)

storage_context = StorageContext.from_defaults(docstore=docstore)

# Create base index
base_index = VectorStoreIndex(hier_nodes, storage_context=storage_context)
base_retriever = base_index.as_retriever(similarity_top_k=6)

# Create auto-merging retriever
auto_merging_retriever = AutoMergingRetriever(
    base_retriever, 
    storage_context,
    verbose=True
)

query = DEMO_QUERIES["advanced"]  # "How do neural networks work in deep learning?"
nodes = auto_merging_retriever.retrieve(query)

print(f"Query: {query}")
print(f"Auto-merged to {len(nodes)} nodes")
for i, node in enumerate(nodes[:3], 1):
    print(f"{i}. Score: {node.score:.4f}" if hasattr(node, 'score') and node.score else f"{i}. (Auto-merged)")
    print(f"   Text: {node.text[:120]}...")
    print()

4. AUTO MERGING RETRIEVER
> Merging 1 nodes into parent node.
> Parent node id: a10042cb-0551-4144-86eb-08e451099963.
> Parent node text: Supervised learning uses labeled training data to learn a mapping from inputs to outputs.

> Merging 1 nodes into parent node.
> Parent node id: a3774f1c-8712-4ad7-8bf6-70f54c7d8478.
> Parent node text: Supervised learning uses labeled training data to learn a mapping from inputs to outputs.

Query: How do neural networks work in deep learning?
Auto-merged to 2 nodes
1. Score: 0.8570
   Text: Deep learning uses neural networks with multiple layers to model and understand complex patterns in data....

2. Score: 0.6956
   Text: Supervised learning uses labeled training data to learn a mapping from inputs to outputs....



## 5. Recursive Retriever – Following Connections Across Content


The Recursive Retriever is built for scenarios where information is interconnected through explicit relationships or references.  
Instead of treating each document or chunk in isolation, it actively follows links — such as citations, cross-references, or hierarchical pointers — to discover and retrieve related content across multiple pieces or layers.

### Core Operating Principle

The retriever works by:

1. Starting with an initial query against a set of nodes
2. Detecting any **reference links** stored in node metadata
3. Automatically following those references to other nodes (or even other retrievers/query engines)
4. Executing sub-queries as needed on the referenced content
5. Combining results from the original matches + all followed references

This creates a traversal mechanism that can jump between related pieces of information, building a more complete picture.

### Types of References It Can Follow

- **Structural / Hierarchical references**  
  Example: small detail chunks pointing to their containing parent section or full document

- **Citation-style references**  
  Example: academic paper excerpts that cite other papers, sections, or figures

- **Metadata-driven links**  
  Example: a generated summary/question that points back to the full source content

- **Cross-document relationships**  
  Example: knowledge graph–like connections between different reports, manuals, or database entries

### Ideal Use Cases

This retriever excels in environments where knowledge is naturally networked:

- Academic literature and research papers with extensive citation networks
- Technical documentation containing many internal/external cross-references
- Legal corpora with references to statutes, precedents, and clauses
- Enterprise knowledge bases with interconnected policies, procedures, and FAQs
- Systems that include tables, figures, appendices, or supplementary materials that are referenced from the main text
- Any domain where understanding requires following chains of related information

### Typical Configuration Elements

- `retriever_map` — dictionary connecting reference keys/IDs to specific retriever instances
- `query_engine_map` — mapping that allows sub-queries to be routed to specialized query engines
- Node metadata fields containing reference pointers (often as node IDs, file paths, or external keys)
- Depth/recursion limits (to prevent runaway reference chasing)

### Primary Strengths

- Naturally handles interconnected knowledge structures
- Enables multi-hop reasoning across documents and references
- Retrieves more complete and contextual answers than isolated chunk retrieval
- Can automatically discover relevant content users didn’t explicitly mention
- Particularly powerful for scholarly, technical, and regulatory domains

### Main Limitations

- Requires accurate, well-structured reference metadata to work effectively
- Setup is significantly more involved than basic retrievers
- Deep or very wide reference chains can become slow and resource-intensive
- Risk of retrieving excessive or tangentially related content if limits aren’t tuned
- Debugging complex traversal paths can be challenging

**In essence**:  
The Recursive Retriever transforms a simple similarity search into an intelligent navigation system that follows meaningful connections — making it the preferred choice whenever your content contains rich relationships, citations, or layered references that should be explored together.

In [10]:
print("=" * 60)
print("5. RECURSIVE RETRIEVER")
print("=" * 60)

# Create documents with references
docs_with_refs = []
for i, doc in enumerate(lab.documents):
    # Add reference metadata
    ref_doc = Document(
        text=doc.text,
        metadata={
            "doc_id": f"doc_{i}",
            "references": [f"doc_{j}" for j in range(len(lab.documents)) if j != i][:2]
        }
    )
    docs_with_refs.append(ref_doc)

# Create index with referenced documents
ref_index = VectorStoreIndex.from_documents(docs_with_refs)

# Create retriever mapping
retriever_dict = {
    f"doc_{i}": ref_index.as_retriever(similarity_top_k=1)
    for i in range(len(docs_with_refs))
}

# Base retriever
base_retriever = ref_index.as_retriever(similarity_top_k=2)

# Add the root retriever to the dictionary
retriever_dict["vector"] = base_retriever

# Recursive retriever
recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict=retriever_dict,
    query_engine_dict={},
    verbose=True
)

query = DEMO_QUERIES["applications"]  # "What are the applications of AI?"
try:
    nodes = recursive_retriever.retrieve(query)
    print(f"Query: {query}")
    print(f"Recursively retrieved {len(nodes)} nodes")
    for i, node in enumerate(nodes[:3], 1):
        print(f"{i}. Score: {node.score:.4f}" if hasattr(node, 'score') and node.score else f"{i}. (Recursive)")
        print(f"   Text: {node.text[:100]}...")
        print()
except Exception as e:
    print(f"Query: {query}")
    print(f"Recursive retriever demo: {str(e)}")
    print("Note: Recursive retriever requires specific node reference setup")
    
    # Fallback to basic retrieval for demonstration
    print("\nFalling back to basic retrieval demonstration...")
    base_nodes = base_retriever.retrieve(query)
    for i, node in enumerate(base_nodes[:2], 1):
        print(f"{i}. Score: {node.score:.4f}")
        print(f"   Text: {node.text[:100]}...")
        print()

5. RECURSIVE RETRIEVER
Retrieving with query id None: What are the applications of AI?
Retrieving text node: Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn from data.
Retrieving text node: Natural language processing enables computers to understand, interpret, and generate human language.
Query: What are the applications of AI?
Recursively retrieved 2 nodes
1. Score: 0.6907
   Text: Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn fr...

2. Score: 0.6500
   Text: Natural language processing enables computers to understand, interpret, and generate human language....



## 6. Query Fusion Retriever – Ensemble Retrieval with Smart Result Blending


The Query Fusion Retriever (often implemented via `QueryFusionRetriever` in LlamaIndex) is an advanced ensemble technique that combines strengths from multiple retrieval approaches and/or multiple reformulations of the original query.  
It boosts overall recall and robustness by intelligently merging results using proven fusion algorithms.

### How It Operates

1. **Multi-Retriever Ensemble**  
   Takes a list of different retrievers (e.g., dense vector + BM25 keyword, or vector retrievers from different indexes/chunk sizes) and runs the query (or queries) against each.

2. **Optional Query Expansion**  
   Uses an LLM to automatically generate several alternative phrasings or sub-queries of the original question (e.g., 3–4 variations).  
   This helps when the user’s wording doesn’t perfectly match the stored content.

3. **Result Fusion**  
   Collects nodes from all retriever × query combinations  
   Applies a fusion/reranking strategy to produce a single, re-ranked list of the most relevant nodes  
   Handles deduplication automatically

### Main Fusion Strategies Available

- **Reciprocal Rank Fusion (RRF)**  
  → Classic, score-independent method: sums 1/(rank + constant) across all result lists  
  → Very robust, ignores absolute score differences, focuses purely on relative positions  
  → Most popular default choice

- **Relative Score Fusion**  
  → Normalizes scores within each retriever/result-set (e.g., Min-Max scaling)  
  → Then performs a weighted combination  
  → Preserves confidence differences within each method

- **Distribution-Based Score Fusion**  
  → Normalizes using statistical properties (mean + standard deviation) of each result set  
  → Helps when retrievers produce scores on very different scales

### Ideal Scenarios

Use Query Fusion when you want:

- Best of both worlds: semantic understanding + exact keyword precision
- Robustness against poorly phrased or ambiguous user questions
- Higher recall for complex, multi-faceted, or exploratory queries
- Combining different indexes (e.g., small vs large chunks, different embedding models)
- General-purpose Q&A systems where query formulation varies widely

### Typical Configuration Options

- List of retrievers to combine
- `num_queries` — how many query variations to generate (set to 1 to disable LLM rewriting)
- `similarity_top_k` — final number of nodes to return after fusion
- `mode` — fusion algorithm (`reciprocal_rerank`, `relative_score`, `dist_based_score`)
- `use_async` — enable parallel execution for speed
- Optional custom prompt for query generation

### Primary Advantages

- Significantly improves recall without sacrificing much precision
- Reduces sensitivity to exact query wording
- Leverages complementary strengths of different retrieval methods
- Automatically handles node deduplication across sources
- Flexible — works with any combination of retrievers

### Key Drawbacks

- Higher latency and compute cost (multiple retrievals + possible LLM calls)
- Additional expense if using LLM for query variations
- Can occasionally introduce less relevant noise if fusion isn’t perfectly tuned
- More complex to set up, debug, and explain compared to single-retriever approaches

**Bottom line**:  
The Query Fusion Retriever is one of the most powerful "plug-and-play" ways to get state-of-the-art retrieval performance in real-world applications — especially when combining semantic vector search with keyword precision and protecting against variable user phrasing.

In [11]:
print("=" * 60)
print("6. QUERY FUSION RETRIEVER - OVERVIEW")
print("=" * 60)

# Create base retriever
base_retriever = lab.vector_index.as_retriever(similarity_top_k=3)

query = DEMO_QUERIES["comprehensive"]  # "What are the main approaches to machine learning?"
print(f"Query: {query}")
print("QueryFusionRetriever generates multiple query variations and fuses results")
print("using one of three sophisticated fusion modes.")

print("\nOverview of Fusion Modes:")
print("1. RECIPROCAL_RERANK: Uses reciprocal rank fusion (most robust)")
print("2. RELATIVE_SCORE: Preserves score magnitudes (most interpretable)")  
print("3. DIST_BASED_SCORE: Statistical normalization (most sophisticated)")

print("\nDemonstration workflow:")
print("Each subsection below explores one fusion mode in detail with:")
print("- Theoretical explanation of the fusion method")
print("- Live demonstration using QueryFusionRetriever")
print("- Manual implementation showing the underlying mathematics")
print("- Use case recommendations and trade-offs")

print(f"\nUsing consistent test query throughout: '{query}'")
print("This allows direct comparison of how each fusion mode handles the same input.")

print("\nProceed to subsections 6.1, 6.2, and 6.3 for detailed demonstrations...")

6. QUERY FUSION RETRIEVER - OVERVIEW
Query: What are the main approaches to machine learning?
QueryFusionRetriever generates multiple query variations and fuses results
using one of three sophisticated fusion modes.

Overview of Fusion Modes:
1. RECIPROCAL_RERANK: Uses reciprocal rank fusion (most robust)
2. RELATIVE_SCORE: Preserves score magnitudes (most interpretable)
3. DIST_BASED_SCORE: Statistical normalization (most sophisticated)

Demonstration workflow:
Each subsection below explores one fusion mode in detail with:
- Theoretical explanation of the fusion method
- Live demonstration using QueryFusionRetriever
- Manual implementation showing the underlying mathematics
- Use case recommendations and trade-offs

Using consistent test query throughout: 'What are the main approaches to machine learning?'
This allows direct comparison of how each fusion mode handles the same input.

Proceed to subsections 6.1, 6.2, and 6.3 for detailed demonstrations...


### 6.1 Reciprocal Rank Fusion (RRF) – The Most Reliable Fusion Approach

Reciprocal Rank Fusion (RRF) is widely regarded as the most robust and dependable method for combining ranked lists in the QueryFusionRetriever.  
It intelligently merges results from multiple query variations (or multiple retrievers) by focusing purely on **relative ranking positions** rather than raw similarity scores.

### How RRF Works in Practice

1. **Multiple query perspectives**  
   The system creates several reformulated versions of the original question  
   Example: "What are main machine learning methods?" → "ML algorithms overview", "core techniques in machine learning", "approaches to learning from data"

2. **Independent retrieval**  
   Each query variation is sent to the retriever(s), producing its own ranked list of results

3. **Reciprocal rank scoring**  
   For every document that appears in any of the lists:  
   → Take its position (rank) in each list  
   → Compute: `1 / (rank + k)`  
   → Sum these values across all lists where the document appeared

4. **Final ranking**  
   Documents are sorted by their total summed reciprocal rank score (highest first)

**Core mathematical expression**:
```
RRF_score(d) = Σ (1 / (rank_i(d) + k))
```
- `d` = document  
- `rank_i(d)` = position of document d in the ranked list of query variation i  
- `k` = constant (standard value = 60) that prevents division by zero and smooths the influence of very high ranks

### Why RRF Performs So Well for Fusion

- **Completely ignores score scales** — works even when different retrievers or queries produce wildly different numerical score ranges  
- **Downplays outliers** — a document that ranks #1 in one list but #200 in another won’t dominate unfairly  
- **Treats ranking position as the primary signal** — which is often more reliable than absolute similarity values  
- **Very stable across different query phrasings** — reduces sensitivity to how the user originally worded the question  
- **Strong track record** — one of the most battle-tested fusion methods in modern information retrieval

### Best Situations to Choose RRF Mode

RRF is typically the **go-to / default** fusion strategy when:

- You are using multiple query reformulations
- Different retrievers produce results on very different scales
- You want maximum stability and consistency in production
- Query phrasing varies widely between users
- You prefer a method that requires almost no tuning

### Key Strengths

- Extremely robust and predictable behavior
- No need to normalize scores or tune complex weights
- Gracefully handles cases where some query variations return fewer results
- Computationally lightweight compared to score-normalization approaches
- Rarely produces pathological results

### Main Trade-offs

- Discards absolute confidence information (only cares about order)
- Gives equal influence to every query variation (no built-in weighting)
- May under-emphasize documents that have very strong scores in one view but mediocre ranks in others
- Less sophisticated than methods that preserve score magnitude when all retrievers are very well calibrated

**In summary**:  
Reciprocal Rank Fusion is the safest, most reliable, and most widely recommended way to combine results in a Query Fusion Retriever — especially when you want consistent performance with minimal tuning and maximum tolerance to variation in query formulation or retriever behavior.

*Based on: https://docs.llamaindex.ai/en/stable/examples/retrievers/reciprocal_rerank_fusion/*


In [12]:
print("=" * 60)
print("6.1 RECIPROCAL RANK FUSION MODE DEMONSTRATION")
print("=" * 60)

# Create QueryFusionRetriever with RRF mode
base_retriever = lab.vector_index.as_retriever(similarity_top_k=5)

print("Testing QueryFusionRetriever with reciprocal_rerank mode:")
print("This demonstrates how RRF works within the query fusion framework")

# Use the same query for consistency across all fusion modes
query = DEMO_QUERIES["comprehensive"]  # "What are the main approaches to machine learning?"

try:
    # Create query fusion retriever with RRF mode
    rrf_query_fusion = QueryFusionRetriever(
        [base_retriever],
        similarity_top_k=3,
        num_queries=3,
        mode="reciprocal_rerank",
        use_async=False,
        verbose=True
    )
    
    print(f"\nQuery: {query}")
    print("QueryFusionRetriever will:")
    print("1. Generate query variations using LLM")
    print("2. Retrieve results for each variation")
    print("3. Apply Reciprocal Rank Fusion")
    
    nodes = rrf_query_fusion.retrieve(query)
    
    print(f"\nRRF Query Fusion Results:")
    for i, node in enumerate(nodes, 1):
        print(f"{i}. Final RRF Score: {node.score:.4f}")
        print(f"   Text: {node.text[:100]}...")
        print()
    
    print("RRF Benefits in Query Fusion Context:")
    print("- Automatically handles query variations of different quality")
    print("- No bias toward queries that return higher raw scores")
    print("- Stable performance across diverse query formulations")
    
except Exception as e:
    print(f"QueryFusionRetriever error: {e}")
    print("Demonstrating RRF concept manually with query variations...")
    
    # Manual demonstration with query variations derived from the main query
    query_variations = [
        DEMO_QUERIES["comprehensive"],  # Original query
        "machine learning approaches and methods",
        "different ML techniques and algorithms"
    ]
    
    print("Manual RRF with Query Variations:")
    all_results = {}
    
    for i, query_var in enumerate(query_variations):
        print(f"\nQuery variation {i+1}: {query_var}")
        nodes = base_retriever.retrieve(query_var)
        
        # Apply RRF scoring
        for rank, node in enumerate(nodes):
            node_id = node.node.node_id
            if node_id not in all_results:
                all_results[node_id] = {
                    'node': node,
                    'rrf_score': 0,
                    'query_ranks': []
                }
            
            # Calculate RRF contribution: 1 / (rank + k)
            k = 60  # Standard RRF parameter
            rrf_contribution = 1.0 / (rank + 1 + k)
            all_results[node_id]['rrf_score'] += rrf_contribution
            all_results[node_id]['query_ranks'].append((i, rank + 1))
    
    # Sort by final RRF score
    sorted_results = sorted(
        all_results.values(), 
        key=lambda x: x['rrf_score'], 
        reverse=True
    )
    
    print(f"\nCombined RRF Results (top 3):")
    for i, result in enumerate(sorted_results[:3], 1):
        print(f"{i}. Final RRF Score: {result['rrf_score']:.4f}")
        print(f"   Query ranks: {result['query_ranks']}")
        print(f"   Text: {result['node'].text[:100]}...")
        print()
    
    print("RRF Formula Demonstration:")
    print("For each document: RRF_score = Σ(1 / (rank + 60))")
    print("- Rank 1 in query: 1/(1+60) = 0.0164")
    print("- Rank 2 in query: 1/(2+60) = 0.0161")
    print("- Rank 3 in query: 1/(3+60) = 0.0159")
    print("Documents appearing in multiple queries get higher combined scores")

6.1 RECIPROCAL RANK FUSION MODE DEMONSTRATION
Testing QueryFusionRetriever with reciprocal_rerank mode:
This demonstrates how RRF works within the query fusion framework

Query: What are the main approaches to machine learning?
QueryFusionRetriever will:
1. Generate query variations using LLM
2. Retrieve results for each variation
3. Apply Reciprocal Rank Fusion
Generated queries:
Here are two search queries related to the input query:
1. "Machine learning methodologies and techniques"
2. "Approaches to machine learning in artificial intelligence"

RRF Query Fusion Results:
1. Final RRF Score: 0.0492
   Text: Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn fr...

2. Final RRF Score: 0.0487
   Text: Deep learning uses neural networks with multiple layers to model and understand complex patterns in ...

3. Final RRF Score: 0.0484
   Text: Supervised learning uses labeled training data to learn a mapping from inputs to outputs....

RRF Ben

### 6.2 Relative Score Fusion Mode – Score-Aware Result Blending

Relative Score Fusion is a sophisticated fusion strategy in the `QueryFusionRetriever` that goes beyond pure ranking positions by **incorporating the actual similarity/confidence scores** produced by each retriever or query variation.  
It normalizes those scores within each result set and then combines them — making it particularly useful when the magnitude of scores carries meaningful information.

### How Relative Score Fusion Operates

1. **Multiple perspectives (optional)**  
   If enabled, an LLM generates several rephrased versions of the original query.

2. **Independent retrieval**  
   Each retriever (or each query variation) returns its top results along with their raw similarity scores.

3. **Per-set normalization**  
   For every individual result list (one per retriever or per query variation):  
   - Apply **Min-Max scaling** to transform the raw scores into a consistent range, usually [0, 1]  
   - The highest-scoring result in that list gets normalized to 1.0  
   - All others are scaled proportionally relative to that maximum

4. **Score combination**  
   For each candidate node:  
   - Sum (or weighted sum) its normalized scores across all lists where it appeared  
   - Final ranking is based on this combined normalized score

**Simplified mathematical view**:
```
normalized_score_i(d) = score_i(d) / max_score_i
combined_score(d) = Σ (weight_i × normalized_score_i(d))
```
(where weights are equal by default, or can be customized via `retriever_weights`)

### Why Choose Relative Score Fusion?

- **Keeps score confidence information** — unlike RRF, it respects how strongly each retriever/query believed in its top results  
- **Handles different scoring scales fairly** — normalization makes scores from vector search, BM25, etc., comparable  
- **More nuanced ranking** — documents with very high confidence in one view can stand out more than in pure rank-based methods  
- **Customizable influence** — you can assign different weights to different retrievers (e.g., favor vector over keyword search)

### Best Situations to Select Relative Score Mode

Use this fusion method when:

- Combining retrievers that produce meaningful similarity scores (dense embeddings + sparse keyword, etc.)
- You want the strength of matches (not just order) to influence final ranking
- Different retrieval methods have very different raw score ranges
- You need more interpretable fusion where score magnitude matters
- Working in hybrid search setups (vector + keyword) where confidence should carry weight

### Configuration in QueryFusionRetriever

- `mode="relative_score"` → activates this fusion algorithm  
- `retriever_weights` — list of floats summing to 1 (e.g. `[0.7, 0.3]` for vector-heavy bias)  
- `similarity_top_k` — final number of nodes after fusion  
- `num_queries` — set >1 to enable LLM query variations (set to 1 to disable)  
- `use_async=True` — recommended for performance

### Primary Advantages

- Captures relative confidence within each retrieval method  
- Produces intuitive, score-proportional final rankings  
- Excellent for hybrid retrieval (semantic + lexical)  
- Allows explicit control over retriever importance via weights  
- Generally more precise than rank-only methods when scores are reliable

### Main Limitations

- Sensitive to outliers — an abnormally high raw score in one list can skew normalization  
- Assumes raw scores are meaningful and comparable after normalization  
- Less robust than RRF when score scales are wildly inconsistent or noisy  
- Slightly more computation than pure reciprocal rank approaches

**Bottom line**:  
Relative Score Fusion is the preferred choice when you want to leverage **both ranking order and score magnitude** for more accurate and confidence-aware result blending — especially in hybrid search setups where different retrievers provide complementary signals with interpretable scores.

*Based on: https://docs.llamaindex.ai/en/stable/examples/retrievers/relative_score_dist_fusion/*

In [13]:
print("=" * 60)
print("6.2 RELATIVE SCORE FUSION MODE DEMONSTRATION")
print("=" * 60)

base_retriever = lab.vector_index.as_retriever(similarity_top_k=5)

print("Testing QueryFusionRetriever with relative_score mode:")
print("This mode preserves score magnitudes while normalizing across query variations")

# Use the same query for consistency across all fusion modes
query = DEMO_QUERIES["comprehensive"]  # "What are the main approaches to machine learning?"

try:
    # Create query fusion retriever with relative score mode
    rel_score_fusion = QueryFusionRetriever(
        [base_retriever],
        similarity_top_k=3,
        num_queries=3,
        mode="relative_score",
        use_async=False,
        verbose=False
    )
    
    print(f"\nQuery: {query}")
    print("QueryFusionRetriever with relative_score will:")
    print("1. Generate query variations")
    print("2. Normalize scores within each variation (score/max_score)")
    print("3. Combine normalized scores")
    
    nodes = rel_score_fusion.retrieve(query)
    
    print(f"\nRelative Score Fusion Results:")
    for i, node in enumerate(nodes, 1):
        print(f"{i}. Combined Relative Score: {node.score:.4f}")
        print(f"   Text: {node.text[:100]}...")
        print()
    
    print("Relative Score Benefits in Query Fusion:")
    print("- Preserves confidence information from embedding model")
    print("- Ensures fair contribution from each query variation")
    print("- More interpretable than rank-only methods")
    
except Exception as e:
    print(f"QueryFusionRetriever error: {e}")
    print("Demonstrating Relative Score concept manually...")
    
    # Manual demonstration with query variations derived from the main query
    query_variations = [
        DEMO_QUERIES["comprehensive"],  # Original query
        "machine learning approaches and methods",
        "different ML techniques and algorithms"
    ]
    
    print("Manual Relative Score Fusion with Query Variations:")
    all_results = {}
    query_max_scores = []
    
    # Step 1: Get results and find max scores for each query
    for i, query_var in enumerate(query_variations):
        print(f"\nQuery variation {i+1}: {query_var}")
        nodes = base_retriever.retrieve(query_var)
        scores = [node.score or 0 for node in nodes]
        max_score = max(scores) if scores else 1.0
        query_max_scores.append(max_score)
        
        print(f"Max score for this query: {max_score:.4f}")
        
        # Store results with normalization info
        for node in nodes:
            node_id = node.node.node_id
            original_score = node.score or 0
            normalized_score = original_score / max_score if max_score > 0 else 0
            
            if node_id not in all_results:
                all_results[node_id] = {
                    'node': node,
                    'combined_score': 0,
                    'contributions': []
                }
            
            all_results[node_id]['combined_score'] += normalized_score
            all_results[node_id]['contributions'].append({
                'query': i,
                'original': original_score,
                'normalized': normalized_score
            })
    
    # Step 2: Sort by combined relative score
    sorted_results = sorted(
        all_results.values(),
        key=lambda x: x['combined_score'],
        reverse=True
    )
    
    print(f"\nCombined Relative Score Results (top 3):")
    for i, result in enumerate(sorted_results[:3], 1):
        print(f"{i}. Combined Score: {result['combined_score']:.4f}")
        print(f"   Score breakdown:")
        for contrib in result['contributions']:
            print(f"     Query {contrib['query']}: {contrib['original']:.3f} → {contrib['normalized']:.3f}")
        print(f"   Text: {result['node'].text[:100]}...")
        print()
    
    print("Relative Score Normalization Process:")
    print("1. For each query variation, find max_score")
    print("2. Normalize: normalized_score = original_score / max_score")
    print("3. Sum normalized scores across all query variations")
    print("4. Documents with consistently high scores across queries win")

6.2 RELATIVE SCORE FUSION MODE DEMONSTRATION
Testing QueryFusionRetriever with relative_score mode:
This mode preserves score magnitudes while normalizing across query variations

Query: What are the main approaches to machine learning?
QueryFusionRetriever with relative_score will:
1. Generate query variations
2. Normalize scores within each variation (score/max_score)
3. Combine normalized scores

Relative Score Fusion Results:
1. Combined Relative Score: 1.0000
   Text: Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn fr...

2. Combined Relative Score: 0.3905
   Text: Supervised learning uses labeled training data to learn a mapping from inputs to outputs....

3. Combined Relative Score: 0.3317
   Text: Deep learning uses neural networks with multiple layers to model and understand complex patterns in ...

Relative Score Benefits in Query Fusion:
- Preserves confidence information from embedding model
- Ensures fair contribution from 

### 6.3 Distribution-Based Score Fusion Mode – Statistically Sophisticated Result Combination

Distribution-Based Score Fusion represents the most statistically aware approach available in the `QueryFusionRetriever`.  
It goes beyond simple rank-based or min-max normalization by actively analyzing the **shape and characteristics** of the score distribution produced by each query variation or retriever, then using that information for smarter, more robust score normalization and combination.

### How Distribution-Based Fusion Works

1. **Multi-perspective retrieval**  
   (Optional) LLM generates several rephrased versions of the query  
   Each version (or each retriever) produces its own ranked list with raw similarity scores

2. **Distribution analysis per result set**  
   For every individual result list, the system computes key statistical properties:  
   - Mean and standard deviation  
   - Median, quartiles, interquartile range (IQR)  
   - Minimum/maximum values, skewness, etc.

3. **Advanced normalization techniques**  
   Common methods applied include:  
   - **Z-score normalization** → `(score - mean) / std_dev`  
     Often followed by a sigmoid transformation to map into a stable [0,1] range  
   - **Percentile-based scaling** → converting each score to its percentile rank within its own list  
   - **Robust scaling using IQR** → especially helpful when distributions contain outliers  
   - **Winsorization** or clipping of extreme values (in some implementations)

4. **Confidence-aware combination**  
   Normalized scores are then aggregated, often with:  
   - Weights influenced by distribution characteristics (e.g., tighter distributions = higher confidence)  
   - Adaptive blending that respects how reliable each query variation appears statistically

### Why This Approach Stands Out

- **Respects real distribution shapes** — handles cases where one query variation produces tight, confident scores while another is scattered  
- **Excellent outlier resistance** — statistical methods naturally dampen the effect of anomalous high/low scores  
- **Adaptive behavior** — can give more influence to query variations that show more consistent, trustworthy score patterns  
- **Works with multi-modal / heterogeneous scoring** — when different retrievers or query phrasings produce fundamentally different score behaviors

### Best Situations to Choose Distribution-Based Mode

Use this fusion strategy when:

- Different query reformulations or retrievers generate dramatically different score distributions  
- Some query variations are expected to be much more reliable than others  
- You’re dealing with noisy retrieval systems or inconsistent scoring mechanisms  
- You want the most statistically principled way to combine potentially unreliable signals  
- Working in research-grade or high-stakes retrieval applications where robustness matters most

### Configuration in QueryFusionRetriever

- `mode="dist_based_score"` (or equivalent identifier in current implementation)  
- Usually automatic distribution analysis and robust scaling  
- Optional parameters for: outlier thresholds, minimum result count for reliable stats, confidence weighting factors  
- Works best when each result list contains enough documents (typically ≥10–20)

### Primary Advantages

- Most mathematically rigorous and theoretically sound fusion method  
- Superior handling of score variability, noise, and outliers  
- Naturally adapts to differences between query variations/retrievers  
- Provides the strongest protection against pathological scoring behavior  
- Especially valuable in hybrid or multi-retriever setups

### Main Drawbacks

- Highest computational cost among the fusion methods  
- Needs sufficient results per query variation to estimate distributions reliably  
- Can be overkill (and slower) for simple or well-behaved scoring scenarios  
- Results are harder to interpret intuitively compared to RRF or relative score methods  
- Requires careful tuning in edge cases with very small result sets

**In summary**:  
Distribution-Based Score Fusion is the most advanced and statistically sophisticated option in the QueryFusionRetriever toolkit — ideal when you need maximum robustness against wildly varying score distributions and want the fusion process to intelligently respect the statistical reliability of each retrieval perspective.



In [14]:
print("=" * 60)
print("6.3 DISTRIBUTION-BASED SCORE FUSION MODE DEMONSTRATION")
print("=" * 60)

base_retriever = lab.vector_index.as_retriever(similarity_top_k=8)

print("Testing QueryFusionRetriever with dist_based_score mode:")
print("This mode uses statistical analysis for the most sophisticated score fusion")

# Use the same query for consistency across all fusion modes
query = DEMO_QUERIES["comprehensive"]  # "What are the main approaches to machine learning?"

try:
    # Create query fusion retriever with distribution-based mode
    dist_fusion = QueryFusionRetriever(
        [base_retriever],
        similarity_top_k=3,
        num_queries=3,
        mode="dist_based_score",
        use_async=False,
        verbose=False
    )
    
    print(f"\nQuery: {query}")
    print("QueryFusionRetriever with dist_based_score will:")
    print("1. Generate query variations")
    print("2. Analyze score distributions for each variation")
    print("3. Apply statistical normalization (z-score, percentiles)")
    print("4. Combine with distribution-aware weighting")
    
    nodes = dist_fusion.retrieve(query)
    
    print(f"\nDistribution-Based Fusion Results:")
    for i, node in enumerate(nodes, 1):
        print(f"{i}. Statistically Normalized Score: {node.score:.4f}")
        print(f"   Text: {node.text[:100]}...")
        print()
    
    print("Distribution-Based Benefits in Query Fusion:")
    print("- Accounts for score distribution differences between query variations")
    print("- Statistically robust against outliers and noise")
    print("- Adapts weighting based on query variation reliability")
    
except Exception as e:
    print(f"QueryFusionRetriever error: {e}")
    print("Demonstrating Distribution-Based concept manually...")
    
    if not SCIPY_AVAILABLE:
        print("⚠️ Full statistical analysis requires scipy")
    
    # Manual demonstration with query variations derived from the main query
    query_variations = [
        DEMO_QUERIES["comprehensive"],  # Original query
        "machine learning approaches and methods",
        "different ML techniques and algorithms"
    ]
    
    print("Manual Distribution-Based Fusion with Query Variations:")
    all_results = {}
    variation_stats = []
    
    # Step 1: Collect results and analyze distributions
    for i, query_var in enumerate(query_variations):
        print(f"\nQuery variation {i+1}: {query_var}")
        nodes = base_retriever.retrieve(query_var)
        scores = [node.score or 0 for node in nodes]
        
        # Calculate distribution statistics
        mean_score = np.mean(scores) if scores else 0
        std_score = np.std(scores) if len(scores) > 1 else 1
        min_score = np.min(scores) if scores else 0
        max_score = np.max(scores) if scores else 1
        
        stats_info = {
            'mean': mean_score,
            'std': std_score,
            'min': min_score,
            'max': max_score,
            'nodes': nodes,
            'scores': scores
        }
        variation_stats.append(stats_info)
        
        print(f"Distribution stats: mean={mean_score:.3f}, std={std_score:.3f}")
        print(f"Score range: [{min_score:.3f}, {max_score:.3f}]")
        
        # Apply z-score normalization
        for node, score in zip(nodes, scores):
            node_id = node.node.node_id
            
            # Z-score normalization
            if std_score > 0:
                z_score = (score - mean_score) / std_score
            else:
                z_score = 0
            
            # Convert to [0,1] using sigmoid
            normalized_score = 1 / (1 + np.exp(-z_score))
            
            if node_id not in all_results:
                all_results[node_id] = {
                    'node': node,
                    'combined_score': 0,
                    'contributions': []
                }
            
            all_results[node_id]['combined_score'] += normalized_score
            all_results[node_id]['contributions'].append({
                'query': i,
                'original': score,
                'z_score': z_score,
                'normalized': normalized_score
            })
    
    # Step 2: Sort by combined distribution-based score
    sorted_results = sorted(
        all_results.values(),
        key=lambda x: x['combined_score'],
        reverse=True
    )
    
    print(f"\nCombined Distribution-Based Results (top 3):")
    for i, result in enumerate(sorted_results[:3], 1):
        print(f"{i}. Combined Score: {result['combined_score']:.4f}")
        print(f"   Statistical breakdown:")
        for contrib in result['contributions']:
            print(f"     Query {contrib['query']}: {contrib['original']:.3f} → "
                  f"z={contrib['z_score']:.2f} → {contrib['normalized']:.3f}")
        print(f"   Text: {result['node'].text[:100]}...")
        print()
    
    print("Distribution-Based Process:")
    print("1. Calculate mean and std for each query variation")
    print("2. Z-score normalize: z = (score - mean) / std")
    print("3. Sigmoid transform: normalized = 1 / (1 + exp(-z))")
    print("4. Sum normalized scores across variations")
    print("5. Results reflect statistical significance across all query forms")

# Show fusion mode comparison summary
print("\n" + "=" * 60)
print("FUSION MODES COMPARISON SUMMARY")
print("=" * 60)
print("All three modes tested with the same query for direct comparison:")
print(f"Query: {query}")
print()
print("Mode Characteristics:")
print("• RRF (reciprocal_rerank): Most robust, rank-based, scale-invariant")
print("• Relative Score: Preserves confidence, normalizes by max score")  
print("• Distribution-Based: Most sophisticated, statistical normalization")
print()
print("Choose based on your use case:")
print("- Production stability → RRF")
print("- Score interpretability → Relative Score")
print("- Statistical robustness → Distribution-Based")

6.3 DISTRIBUTION-BASED SCORE FUSION MODE DEMONSTRATION
Testing QueryFusionRetriever with dist_based_score mode:
This mode uses statistical analysis for the most sophisticated score fusion

Query: What are the main approaches to machine learning?
QueryFusionRetriever with dist_based_score will:
1. Generate query variations
2. Analyze score distributions for each variation
3. Apply statistical normalization (z-score, percentiles)
4. Combine with distribution-aware weighting

Distribution-Based Fusion Results:
1. Statistically Normalized Score: 0.8696
   Text: Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn fr...

2. Statistically Normalized Score: 0.5908
   Text: Supervised learning uses labeled training data to learn a mapping from inputs to outputs....

3. Statistically Normalized Score: 0.5646
   Text: Deep learning uses neural networks with multiple layers to model and understand complex patterns in ...

Distribution-Based Benefits in